## CloudDBH

In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import os
import gc
import open3d as o3d

from shapely.ops import cascaded_union, polygonize
from shapely.geometry import Polygon, LineString
import shapely.geometry as geometry

from scipy import optimize
from scipy.spatial import ConvexHull, convex_hull_plot_2d, Delaunay
import scipy.io as sio

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import math
from math import pi

from sklearn.neighbors import NearestNeighbors
from descartes import PolygonPatch


# Functions

### From Calders et al. 

In [2]:
# Functions from Calders et al
# nearest neighbour needed for outlier removal
def nn(x,rad):
    """
    Function from TLS_Inventory.
    
    x: a numpy array, rad: radius to search for neighbors within
    
    My understanding - can either specify number of neighbors to return (in sklearn NearestNeighbors),
    or return all the neighbors and then filter to return only those within a specified distance radius
    """
    nbrs = NearestNeighbors(algorithm='auto', metric='euclidean').fit(x) 
    distances, indices = nbrs.radius_neighbors(radius=rad) #find all neighbours within this search radius
    
    #output explained: https://stackoverflow.com/questions/53886289/confused-with-the-output-of-sklearn-neighbors-nearestneighbors 
    return distances, indices


def load_pts(cloud,downsample=False,outliers=False): 
    """
    Function from TLS_Inventory that reads the point cloud data for single tree at a time and
    returns pandas dataframe with the points split into x, y, z coordinates. 
    
    cloud: point cloud from a single tree - note, point cloud data should be pre-processed and 
           split into individual trees prior to using this function.
    downsample (Optional): Default is False, meaning all the points are retained. If 
                           True, the point clouds are sampled to keep only fraction of points.
    outliers (Optional): Default is False, meaning outliers are retained. If True, 
    """
    # read the point cloud data for single tree and save as pandas df with columns for x, y, z coords
    dftemp=o3d.io.read_point_cloud(cloud)
    df=pd.DataFrame(data=np.asarray(dftemp.points),columns=['x', 'y', 'z']) #access the points
    
    # Optional downsampling
    if downsample:
        df=df.sample(frac=0.1) # keep 10pct of points
    
    # Optional outlier removal 
    if outliers: #remove outliers
        xy=df.iloc[:,0:2].values # takes ALL rows, first and second column - x and y - and .values converts to numpy array
        dist, indi = nn(xy,0.5) # get nearest neighbors within search radius of 0.5
        cnt=[len(i) for i in indi] # count the kNN within the search radius
        cnt = pd.DataFrame({'count':cnt})
        
        # set threshold for the number of neighbors we want to keep 
        threshold=df.shape[0]*0.0001 #1 neighbor for every 10 000 pts
        
        removed=sum(np.array(cnt)<threshold)
        df=df[np.array(cnt)>=threshold]
        print("Removed %i outliers using kNN threshold %.2f" % (removed[0], threshold-1))
    
    return df #return pandas dataframe 

### Helper Functions

In [3]:
# TODO: dont use this format plots function find a package
def formatplots(trees, ncols=4, shape='square'):
    # determine number of subplots
    nplots = len(trees)
    
    # determine number of rows 
    if(nplots % ncols) != 0:
        nrows = int(nplots / ncols + 1)
    else:
        nrows = int(nplots / ncols)

    if shape == 'horiz':
        # initiate figure with nrows and ncols
        fig, axs = plt.subplots(nrows, ncols, figsize=(20,nrows*4))
        fig.tight_layout()
    if shape == 'vert':
        # initiate figure with nrows and ncols
        fig, axs = plt.subplots(nrows, ncols, figsize=(20,nrows*10))
        fig.tight_layout()
    if shape == 'square':
        # initiate figure with nrows and ncols
        fig, axs = plt.subplots(nrows, ncols, figsize=(20,nrows*10))
        fig.tight_layout()
    
    return fig, axs

In [4]:
def update_inds(treeind, rowind, colind, ncols):
    # increase tree index to get to next tree
    treeind += 1

    if (treeind % ncols) != 0:
        colind += 1
    else:
        rowind +=1 
        colind = 0
    
    return treeind, colind, rowind

In [5]:
def construct_treeid(tree, ptspath):
    # For wytham woods tree dataset
    if str(ptspath).__contains__('DATA_clouds_ply'):
        if tree[-4:] != '.ply':
            tree = str(ptspath) + '/wytham_winter_' + str(tree) + '.ply' # TODO: don't hard code this
        else:
            tree = tree # TODO: make this more robust

        tree_num = tree.split("_")[-1].split(".")[0]
    
    # For tropical tree dataset - must have the leaf files already removed
    if str(ptspath).__contains__('Tropical_manual_ply'):
        if tree[-4:] != '.ply':
            tree = str(ptspath) + '/MLA01_2018_' + str(tree) + '.wood.ply' # TODO: don't hard code this
        else:
            tree = str(tree) # TODO: make this more robust

        tree_num = tree.split("_")[-1].split(".")[0]
    return tree, tree_num

In [6]:
def wytham_singlemulti(ptspath):
    """
    Function that splits the wytham woods dataset into single vs multistemmed trees 
    with string operations based on how the wytham dataset is named.
    """

    singlestemmed = []
    multistemmed = []
    
    # get list of trees from pts path
    trees = glob.glob("%s/*ply" % pts_path)
    
    for tree in trees:
        tree_num = tree.split("_")[-1].split(".")[0]
        
        # if the treeID ends in a letter, it is not single stem
        if tree_num[-1].isalpha():
            multistemmed.append(tree)
        else:
            singlestemmed.append(tree)

    return singlestemmed, multistemmed

### Tree Functions

In [22]:
def calc_convexhull(tree, ptspath, hgt=1.3):
    """
    Function to calculate and plot convex hulls for each tree in a list of trees.
    
    Returns: 
        dbh - value for diameter at the specified height 
        verts - number of vertices used in calculating convex hull 
        pts_dbh - the 3D point dataset for the given slice (for plotting)
    """

    # Get treeID and ensure inputted format is correct
    tree, tree_num = construct_treeid(tree, ptspath)

    # get DBH from full cloud to preserve max number of hits on stem slice
    pts = load_pts(tree,False,False)

    # extract part of df with z values between 1.27 and 1.33 m by default - Tansey et al. 2009, Calders et al. 2015
    pts_dbh = pts[(pts['z'] > pts['z'].min() + hgt - 0.03) & (pts['z'] < pts['z'].min() + hgt + 0.03)]
    xy_dbh_arr = np.asarray(pts_dbh[['x', 'y']])

    # Calculate DBH (convex hull) using scipy ConvexHull
    circum = ConvexHull(xy_dbh_arr)
    
    # calculate return values
    dbh = circum.area / np.pi
    verts = len(circum.vertices)
                      
    return dbh, verts, pts_dbh
    

In [76]:
# TODO: redo this function so it takes in better args

def plot_convexhull(treelist, ptspath, ncols=4, fixedheight = True, hgt=1.3, hgtlist = None):
    fig, axs = formatplots(treelist, ncols)
    rowind = colind = treeind = 0
    
    for tree in treelist:
        tree, tree_num = construct_treeid(tree, ptspath)
        
        if fixedheight:
            hgt = hgt
        else:
            hgt = hgtlist[tree_num]
        
        dbh_cv, verts, pts_dbh, dbh = calc_convexhull(tree, pts_path, hgt=hgt)
        xy_dbh_arr = np.asarray(pts_dbh[['x', 'y']])
        
        
        axs[rowind, colind].scatter(pts_dbh['x'], pts_dbh['y']) 
        
        # plot line connecting all the dbh vertices
        for simplex in dbh.simplices:
            axs[rowind, colind].plot(xy_dbh_arr[simplex, 0], xy_dbh_arr[simplex, 1], 'r--')
        
        axs[rowind, colind].plot(xy_dbh_arr[dbh.vertices,0], xy_dbh_arr[dbh.vertices,1], 'r--', label=f"DBH: {dbh.area:.3f}") # DBH (convex hull)
        axs[rowind, colind].plot(xy_dbh_arr[dbh.vertices,0], xy_dbh_arr[dbh.vertices,1], 'ro', label=f"Vertices: {len(dbh.vertices)}") # vertices

        axs[rowind, colind].set_title(f'DBH (Convex Hull) for {tree_num}')
        axs[rowind, colind].legend(fontsize='xx-large', loc='upper left')
        axs[rowind, colind].axis('equal')

        treeind, colind, rowind = update_inds(treeind, rowind, colind, ncols)
    return

In [71]:
def iter_dbh(tree, ptspath, step=0.1, lim=15):
    """
    Start from bottom of tree and calculate convex hull in 0.06m slices separated
    by defined step size until limit. 
    """

    # Get treeID
    tree, tree_num = construct_treeid(tree, ptspath)

    #get DBH from full cloud to preserve max number of hits on stem slice
    pts = load_pts(tree,False,False)

    # calculate dbh for slices up to smaller of lim or tree height
    disc = 0.1
    hgt = pts['z'].max() - pts['z'].min()
    
    # set the limit equal to the smaller of tree height and defined limit
    lim = min(hgt, lim)
    
    # empty lists to hold values for each tree
    dbh_list = []
    vert_list = []
    disc_list = []

    # loop upward through height of tree (up to lim)
    while disc < lim:
        # Calculate DBH (convex hull) for current slice using scipy ConvexHull
        pts_slice = pts[(pts['z'] > pts['z'].min() + disc - 0.03) & (pts['z'] < pts['z'].min() + disc + 0.03)]
        xy_dbh = np.asarray(pts_slice[['x', 'y']])

        if len(xy_dbh) < 3:
            print(f'tree {tree_num} had not enough points at disc {disc}')
            disc += step
            break

        circum = ConvexHull(xy_dbh) 
        
        dbh = circum.area / np.pi
        verts = len(circum.vertices)

        # append data to lists
        dbh_list.append(dbh)
        vert_list.append(verts)
        disc_list.append(round(disc, 1))

        # move up to next slice  
        disc += step
    
    df_out = pd.DataFrame(list(zip(dbh_list, vert_list, disc_list)), columns = ['DBH_CV', 'DBH_Verts', 'Slice_Hgt'])
    
    return df_out
            

In [ ]:
def plot_iter_dbh(iterdbh_dict, treeslopes, ncols = 2):
    """
    Takes the output of function iter_dbh and plots results in grid.
    """
    
    # sort output of iter_dbh function and get list of unique trees
    #iterdbhout.sort_values('TLS_ID', inplace=True)
    #uniquetrees = iterdbhout.TLS_ID.unique()
    
    uniquetrees = iterdbh_dict.keys()
    print(uniquetrees)
    
    # format plotting grid
    fig, axs = formatplots(uniquetrees, ncols, shape='horiz')
    rowind = colind = treeind = 0

    # initiate figure with nrows and ncols
    #fig, axs = plt.subplots(nrows, ncols, figsize=(40,180), sharex=True)
    #axs[0, 0].set_xlim(right=3)

    for tree in uniquetrees:
        #treedf = iterdbhout.loc[iterdbhout['TLS_ID'] == tree]
        #treedf = iterdbh_dict[tree]
        treedf = treeslopes[tree]
        
        filt = treedf['Flagged'] == 1
        buttress = treedf['Buttress'] == 1

        # calculate trendline on valid data
        # validdf = treedf[~filt]
        
        # plot data
        axs[rowind, colind].scatter(treedf['Slice_Hgt'], treedf['DBH_CV'])
        axs[rowind, colind].set_title(f'DBH w Height for {tree}', fontsize=20)
        axs[rowind, colind].set_xlabel('Height of DBH Slice (m)', fontsize=20)
        axs[rowind, colind].set_ylabel('Value of DBH Slice (m)', fontsize=20)
        axs[rowind, colind].plot(treedf['Slice_Hgt'][filt], treedf['DBH_CV'][filt],'r+', markersize=20)
        axs[rowind, colind].plot(treedf['Slice_Hgt'][buttress], treedf['DBH_CV'][buttress],'b+', markersize=20)
        
        fig.tight_layout()

        
        treeind, colind, rowind = update_inds(treeind, rowind, colind, ncols)
    
    return

In [82]:
def get_buttress_hgt(tree, ptspath, step=0.1, lim=15, thresh = 1.0):
    
    # Get treeID
    tree, tree_num = construct_treeid(tree, ptspath)
    
    # run iterative dbh for specified step size and lim to calc slopes
    treedf = iter_dbh(tree, ptspath, step=step, lim=lim)

    # calculate slope between pair of dbh measurements
    for disc in treedf['Slice_Hgt'].iloc[1:-1]:

        # round disc heights to one decimal place
        disc = round(disc, 1)
        prev_disc = round(disc - step, 1)

        # get dbh values at each disc height
        val1 = treedf[treedf['Slice_Hgt'] == disc]['DBH_CV'].iloc[0]
        val2 = treedf[treedf['Slice_Hgt'] == prev_disc]['DBH_CV'].iloc[0]

        # calculate slope between dbh measurements
        slope = (val1 - val2) / step

        # make new column in treedf for slope
        treedf.loc[treedf['Slice_Hgt'] == prev_disc, 'slope'] = slope

        # flag slope values greater than given threshold
        if abs(slope) > thresh:
            treedf.loc[treedf['Slice_Hgt'] == prev_disc, 'Buttress'] = 1

        else:
            treedf.loc[treedf['Slice_Hgt'] == prev_disc, 'Buttress'] = 0
        
    # if first three points are not flagged as buttress points:
    if all(np.array(treedf['Buttress'].iloc[0:3]) == np.array([0, 0, 0])):
        # set buttress height to 0.0 (i.e. bottom of tree stem)
        buttress_index = 0
        buttress_hgt = 0.0

    # otherwise, find height of top of buttress
    else:
        # find first sequence of three 0's (non-buttress points) 
        val_find = np.array([0, 0, 0])
        df_condition = treedf['Buttress'].rolling(3).apply(lambda g: all(g.astype(int) == val_find), raw=True)

        # get the index where three 0's first occurs and height of buttress
        buttress_index = np.where(df_condition == 1)[0][0]
        buttress_hgt = treedf['Slice_Hgt'].iloc[buttress_index]

               
    return buttress_hgt

In [119]:
# TODO: lim should be set outside functions to be equal to default or tree height, whichever is less
def flag_irregular(tree, ptspath, step=0.1, thresh=1.0, lim=3):
    """
    Flag any irregular parts of the tree stem to avoid using them for dbh.
    
    Using a rolling mean to easily account for/ avoid the influence of extremely 
    small outliers that may be caused by data occlusion. 
    """
    # run iterative dbh for specified step size and lim to calc slopes
    treedf = iter_dbh(tree, ptspath, step=step, lim=lim)
    
    # get buttress height and subset dataframe to parts over the buttress
    buttress_hgt = get_buttress_hgt(tree, ptspath, step=0.1, lim=15, thresh=1.0)
    over_buttress = treedf[treedf["Slice_Hgt"] > buttress_hgt]
    
    # calculate a max dbh threshold - no greater 2 * minimum rolling mean dbh
    dbh_thresh = np.nanmin(over_buttress['DBH_CV'].rolling(3).mean()) * 2
    
    # create a filter using this threshold
    dbh_filt = treedf['DBH_CV'] > dbh_thresh
    
    # flag values greater than the threshold
    treedf.loc[dbh_filt, 'Flagged'] = 1
    treedf.loc[~dbh_filt, 'Flagged'] = 0

    return treedf

In [118]:
def get_dbh_hgt(tree, ptspath):
    """
    Take output of iter_dbh function and determine ideal height from which to calculate dbh
    
    Note: the rolling search doesn't seem to work if there is no buttress so this is broken up into 
    if else - TODO: but could simplify if I figure out how to fix the else part of the code to work
    for non buttressed trees too 
    """
    # Get treeID
    tree, tree_num = construct_treeid(tree, ptspath)

    # get buttress height
    buttress_hgt = get_buttress_hgt(tree, ptspath, step=0.1, lim=15, thresh=1.0)
    
    # flag irregular points to be excluded from dbh height consideration
    treedf = flag_irregular(tree, ptspath, step=0.1, thresh=1.0, lim=15)

    # subset treedf to only above buttress height
    over_buttress = treedf[treedf["Slice_Hgt"] > buttress_hgt]
    
    # select only non-flagged data
    filt = over_buttress['Flagged'] == 1
    validdf = over_buttress[~filt]

    # get suitable value closest to 1.3m to use for dbh height
    disclist = list(validdf['Slice_Hgt'])
    dif = lambda disclist : abs(disclist - 1.3)
    hgt = min(disclist, key=dif)

    return hgt

In [15]:
def plot2D(trees, ptspath, ncols=4, dbh=True, iterdbh=False, hgtlist = None):
    """
    Function to plot tree point clouds in x/z or y/z direction
    
    TODO: make iter dbh function work on just one tree - then call iter_dbh from within this function
    """
    # determine number of rows to plot figures
    fig, axs = formatplots(trees, ncols)
    rowind = colind = treeind = 0

    # loop through trees
    for tree in trees:
        # Get treeID
        tree, tree_num = construct_treeid(tree, ptspath)

        #get DBH from full cloud to preserve max number of hits on stem slice
        pts = load_pts(tree,False,False)

        # calculate tree height
        tree_hgt = pts['z'].max() - pts['z'].min()
            
        max_pt = pts[pts['z'] == pts['z'].max()]

        # plot scatter plot of points in x and z 
        axs[rowind, colind].scatter(pts['x'], pts['z'], s=3) 
        if dbh: 
            pts_dbh = pts[(pts['z'] > pts['z'].min() + 1.27) & (pts['z'] < pts['z'].min() + 1.33)]
            axs[rowind, colind].plot(pts_dbh['x'], pts_dbh['z'], 'ro', markersize=3, label=f"DBH Slice")
        
        if iterdbh: 
            hgt = hgtlist[tree_num]
            pts_hgt = pts[(pts['z'] > pts['z'].min() + hgt - 0.03) & (pts['z'] < pts['z'].min() + hgt + 0.03)]
            axs[rowind, colind].plot(pts_hgt['x'], pts_hgt['z'], 'yo', markersize=3, label=f"HGT Slice")
            

        axs[rowind, colind].annotate(f'Height: {tree_hgt:.2f}', xy=(max_pt['x'].iloc[0], max_pt['z'].iloc[0]), va='top', ha='left', size=15)
        axs[rowind, colind].legend(fontsize='xx-large', loc='upper left')
        axs[rowind, colind].set_title(f'{tree_num} ({rowind}, {colind})')

        # increase tree index to get to next tree
        treeind, colind, rowind = update_inds(treeind, rowind, colind, ncols)
    return



# Main Code

In [16]:
# Wytham Woods
pts_path = "/Users/snix/Documents/2021-2023/UCL/Dissertation/DATA_clouds_ply"
qsm_path = "/Users/snix/Documents/2021-2023/UCL/Dissertation/DATA_QSM_opt/"
tree_csv_path = "/Users/snix/Documents/2021-2023/UCL/Dissertation/Calders Et Al/analysis_and_figures/trees_summary.csv"

# Tropical Trees
trop_pts_path = '/Users/snix/Documents/2021-2023/UCL/Dissertation/Tropical_manual_ply'
trop_csv_path = '/Users/snix/Documents/2021-2023/UCL/Dissertation/MLA01_man.csv'

In [17]:
trees = glob.glob("%s/*ply" % pts_path)
singlestemmed, multistemmed = wytham_singlemulti(pts_path)

print(f'Singlestemmed: {len(singlestemmed)}')
print(f'Multistemmed: {len(multistemmed)}')

Singlestemmed: 585
Multistemmed: 291


In [18]:
trop_trees = glob.glob("%s/*wood.ply" % trop_pts_path)

## Running - New

In [115]:
tree_data = {}

for tree in trop_trees[0:10]:
    tree, tree_num = construct_treeid(tree, trop_pts_path)
    
    dbh, verts, pts_dbh = calc_convexhull(tree, trop_pts_path, hgt=1.3)
    
    hgt = get_dbh_hgt(tree, trop_pts_path)
    
    tree_data[tree_num] = hgt


dbh threshold: 0.2915608991204565
tree T38, res: 1.3
dbh threshold: 0.8433717620074527
tree T39, res: 1.3
dbh threshold: 0.2866808305943029
tree T187, res: 1.3
dbh threshold: 0.19436154555876226
tree T115, res: 1.3
tree T94 had not enough points at disc 13.699999999999967
tree T94 had not enough points at disc 13.699999999999967
tree T94 had not enough points at disc 13.699999999999967
dbh threshold: 0.27856595834116643
tree T94, res: 1.3
dbh threshold: 0.394078567984411
tree T304, res: 1.3
dbh threshold: 0.8739579252501121
tree T95, res: 1.3
dbh threshold: 0.1590663212322125
tree T379, res: 1.3
dbh threshold: 0.30163365519819535
tree T120, res: 1.3
dbh threshold: 0.252252627527427
tree T347, res: 1.3


# Old Code

### Calculate Convex Hull at 1.3:

In [ ]:
datadict = {}

# ENGLAND TREES
for tree in singlestemmed:
    # construct tree id
    tree, tree_num = construct_treeid(tree, pts_path)
    
    # calc convex hull
    dbh_cv, verts, pts_dbh, dbh = calc_convexhull(tree, pts_path, hgt=1.3)
    
    datadict[tree_num] = {'dbh_cv': dbh_cv, 'verts': verts}


# MALAYSIA TREES
datadict2 = {}

for tree in trop_trees:
    # construct tree id
    tree, tree_num = construct_treeid(tree, trop_pts_path)
    
    # calc convex hull
    dbh_cv, verts, pts_dbh, dbh = calc_convexhull(tree, trop_pts_path, hgt=1.3)
    
    datadict2[tree_num] = {'dbh_cv': dbh_cv, 'verts': verts}
    

In [ ]:
plot_convexhull(trop_trees[0:12], trop_pts_path, ncols=4, fixedheight = True)

In [ ]:
datadf = pd.DataFrame.from_dict(datadict2, orient='index')

datadf = datadf.reset_index()
datadf = datadf.rename(columns={"index": "TLS_ID"})


datadf.sort_values('dbh_cv', ascending=False, inplace=True)

datadf.head()

In [ ]:
# appears that larger dbh values are coming from issues with pt clouds
# sort by dbh to highlight trees with issues 
datadf.sort_values('dbh_cv', ascending=False, inplace=True)

# make new list of just the top 12 'large dbh' trees and plot convex hull results
bigtrees = list(datadf.iloc[0:12, 0])

print(bigtrees)

bigtree_datadict = {}

for tree in bigtrees:
    # construct tree id
    tree, tree_num = construct_treeid(tree, trop_pts_path)
    
    # calc convex hull
    dbh_cv, verts, pts_dbh, dbh = calc_convexhull(tree, trop_pts_path, hgt=1.3)
    bigtree_datadict[tree_num] = {'dbh_cv': dbh_cv, 'verts': verts}

plot_convexhull(bigtrees, trop_pts_path, ncols=4, fixedheight = True)

### Iterative DBH by Slice:

In [ ]:
# Create a dictionary with tree ids as keys and values of dataframes of dbh along stem
# TODO: make these not overwrite each other - how should I do tropical and england together :(
iterdbh_dict = {}

# ENGLAND TREES
for tree in singlestemmed:
    tree_num, iterdbhout = iter_dbh(tree, pts_path, step=0.1, lim=3)
    
    iterdbh_dict[tree_num] = iterdbhout
    

# MALAYSIA TREES
iterdbh_dict_2 = {}

for tree in trop_trees:
    tree_num, iterdbhout = iter_dbh(tree, trop_pts_path, step=0.1, lim=15)
    
    iterdbh_dict_2[tree_num] = iterdbhout

In [ ]:
# run for just a few trees to get a dictionary to plot
iterdbh_dict_plot = {}
for tree in bigtrees[5:11]:
    # bump up limit for tropical trees to make sure we get above the buttress
    tree_num, iterdbhout = iter_dbh(tree, trop_pts_path, step=0.1, lim=15)
    
    iterdbh_dict_plot[tree_num] = iterdbhout
    
plot_iter_dbh(iterdbh_dict_plot, ncols = 2)

In [ ]:
# CHECK BUTTRESS
treeslopes = flag_buttress(iterdbh_dict_2)

In [ ]:
treeslopes['T95']

In [ ]:
#nplots = len(treeslopes.keys())
ncols = 4
fig, axs = formatplots(treeslopes.keys(), ncols)
rowind = colind = treeind = 0

plottrees = list(treeslopes.keys())

# loop through trees
for tree in plottrees[0:20]:
    treedf = treeslopes[tree]
    
    axs[rowind, colind].scatter(treedf['Slice_Hgt'], treedf['slope'])

    axs[rowind, colind].set_title(f'{tree} ({rowind}, {colind})')

    # increase tree index to get to next tree
    treeind, colind, rowind = update_inds(treeind, rowind, colind, ncols)

In [ ]:
# run for just a few trees to get a dictionary to plot
iterdbh_dict_plot = {}

for tree in plottrees[0:20]:
    # bump up limit for tropical trees to make sure we get above the buttress
    tree_num, iterdbhout = iter_dbh(tree, trop_pts_path, step=0.1, lim=15)
    
    iterdbh_dict_plot[tree_num] = iterdbhout
    
plot_iter_dbh(iterdbh_dict_plot, treeslopes, ncols = 2)

In [ ]:
dbh_hgts = get_dbh_hgt(iterdbh_dict_plot)

In [ ]:
plot2D(plottrees[0:20], trop_pts_path, ncols=4)

In [ ]:
# get heights at which to record dbhh for full dataset
treedf, dbh_hgts = get_dbh_hgt(iterdbh_dict_2, check_buttress=True)

In [ ]:
treedf.head()

In [ ]:
# now calculate dbh at the specified heights to avoid branches
big_datadict = {}

for tree in singlestemmed:
    # construct tree id
    tree, tree_num = construct_treeid(tree, pts_path)
    
    dbh_hgt = dbh_hgts[tree_num]
    
    # calc convex hull
    dbh_cv, verts, pts_dbh, dbh = calc_convexhull(tree, pts_path, hgt=dbh_hgt)
    
    big_datadict[tree_num] = {'dbh_cv': dbh_cv, 'verts': verts}

plot_convexhull(singlestemmed[0:8], pts_path, ncols=4, fixedheight = False, hgtlist = dbh_hgts)  

In [ ]:
plot2D(singlestemmed[0:8], pts_path, ncols=4, dbh=True, iterdbh=True, hgtlist = dbh_hgts)

In [ ]:
#convert full data dictionary into dataframe for analysis and predictions

big_datadf = pd.DataFrame.from_dict(big_datadict, orient='index')


big_datadf = big_datadf.reset_index()
big_datadf = big_datadf.rename(columns={"index": "TLS_ID"})

big_datadf

# DBH (Convex Hull) Results

### Predict DBH:

In [ ]:
treecsv = pd.read_csv(tree_csv_path)


treecsv = treecsv[['TLS_ID','stemlocx_[m]', 'stemlocy_[m]', 'DBH_TLS_[m]',
                    'Hgt_pts_[m]', 'VerticalCrownProjectedArea_pts_[m2]', 'Vol_QSM_avg_[m3]',
                    'Vol_QSM_D0_25mm_avg_[m3]', 'Vol_QSM_D25_50mm_avg_[m3]', 'Vol_QSM_D50_75mm_avg_[m3]',
                    'Vol_QSM_D75_100mm_avg_[m3]', 'Vol_QSM_D100_200mm_avg_[m3]', 'Vol_QSM_D200_500mm_avg_[m3]',
                    'Vol_QSM_D500_1000mm_avg_[m3]', 'Vol_QSM_D1000mm_avg_[m3]']]


treedata = pd.merge(big_datadf, treecsv, on ='TLS_ID')

treedata.head()

## Compare Convex Hull with "DBH_TLS[m]"

In [ ]:
# select only single stem trees from csv 
#ss_treedata = treecsv[treecsv[['TLS_ID']].apply(lambda x: x[-1].isdigit(), axis=1)]

#smalltrees = treedata[treedata['DBH_CV']<4]

plt.figure(figsize=(10,7))
plt.scatter(treedata['dbh_cv'], treedata['DBH_TLS_[m]'])
plt.xlabel('DBH from Convex Hull', fontsize=20)
plt.ylabel('DBH from TLS', fontsize=20)


## Random Forest (Predicting DBH from QSMs)

In [ ]:
#https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

# Labels are the values we want to predict
labels = np.array(treedata['dbh_cv'])

# Remove the labels from the features
# axis 1 refers to the columns
features = treedata.drop(['TLS_ID','dbh_cv', 'DBH_TLS_[m]'], axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)



In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')



In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
#rf_results = pd.DataFrame(mape, predictions, test_labels)

rf_results = pd.DataFrame({'mape': mape, 'errors': errors,'predictions': predictions, 'test_labels': test_labels}, columns=['mape', 'errors', 'predictions', 'test_labels'])

plt.figure(figsize=(10,7))
plt.scatter(rf_results['predictions'], rf_results['test_labels'])
plt.xlabel('Predictions (DBH)', fontsize=20)
plt.ylabel('Measured Values (DBH)', fontsize=20)


In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(rf_results['test_labels'], rf_results['errors'])
plt.xlabel('Measured Values (DBH)', fontsize=20)
plt.ylabel('Errors', fontsize=20)

In [ ]:
# merge rf results back with original dataset
#https://stackoverflow.com/questions/40729162/merging-results-from-model-predict-with-original-pandas-dataframe
# TODO: in final - re-run model on full dataset and split into training vs testing

df_out = pd.merge(treedata,rf_results[['mape', 'errors', 'predictions']],how = 'left',left_index = True, right_index = True)

#rf_results.shape
df_out.head(150)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(df_out['verts'], df_out['errors'])
plt.xlabel('DBH Verts', fontsize=20)
plt.ylabel('Errors', fontsize=20)

# Old Code: 